In [ ]:
!pip install pyyaml==5.1
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 5.3 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=2ec212d6f5d229f5046ab1e16320b43bcd142a396e7d1e04d745ee144ccf1bc2
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-6pu26_hg
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-6pu26_hg
     |████████████████████████████████| 50 kB 3.1 MB/s 
     |████████████████████████████████| 79 kB 

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize
import os
import torch
import copy
import joblib

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T

from pycocotools.coco import COCO
import skimage.io as io
from detectron2.utils.visualizer import ColorMode

In [ ]:
parts_class = ['1++', '1+', '1', '2', '3']

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import numpy as np
# import cv2

# mask = np.zeros((300,300))
# pts1 = [np.array([(100, 200), (100, 200), (200, 200), (200, 100)], dtype=np.int32)]
# cv2.drawContours(mask, pts1, -1, 2.5, 10)
# mask = mask.astype(np.uint8)

# cv2.findContours(mask, ,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

SyntaxError: ignored

In [ ]:
# import numpy as np

# im = cv2.imread('/content/drive/MyDrive/KoreanBeef/data/01_splitdataset/val/images/cow_1++_100.jpg')
# imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(imgray, 127, 255, 0)
# im2, contours, hierarchy = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# cv.drawContours(img, contours, -1, (0,255,0), 3)
# plt.imshow(im2)

error: ignored

In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import cv2
from tqdm import tqdm
import joblib

def format_mask(mask):
  n_mask = []
  for line_idx in range(len(mask)):
    line_list = list(mask[line_idx])
    for ele_idx in range(len(line_list)):
      if line_list[ele_idx] == 1:
        line_list[ele_idx] = 0
      else:
        line_list[ele_idx] = 1

      if line_idx < 4:
        line_list[ele_idx] = 0
      elif line_idx > len(mask) - 5:
        line_list[ele_idx] = 0
      if ele_idx < 4 or ele_idx > len(line_list) -5 :
        line_list[ele_idx] = 0
    n_mask.append(line_list)
  return n_mask

import skimage.measure as measure

def close_contour(contour):
  if not np.array_equal(contour[0], contour[-1]):
    contour = np.vstack((contour, contour[0]))
  return contour

def binary_mask_to_polygon(binary_mask, tolerance=0):
  polygons = []
  # pad mask to close contours of shapes which start and end at an edge
  padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
  contours = measure.find_contours(padded_binary_mask, 0.5)
  contours = np.subtract(contours, 1)
  for contour in contours:
      contour = close_contour(contour)
      contour = measure.approximate_polygon(contour, tolerance)
      if len(contour) < 3: 
          continue
      contour = np.flip(contour, axis=1)
      segmentation = contour.ravel().tolist()
      # after padding and subtracting 1 we may get -0.5 points in our segmentation
      segmentation = [0 if i < 0 else i for i in segmentation]
      polygons.append(segmentation)

  return polygons


def get_cow_dicts(directory):
    classes = ['1++', '1+', '1', '2', '3']
    dataset_dicts = []
    for idx, filename in enumerate(tqdm([file for file in os.listdir(directory) if file.endswith('.jpg')])):
        img_file = os.path.join(directory, filename)
        img = cv2.imread(img_file, cv2.IMREAD_COLOR)
        height, width, c = img.shape
        tclass = filename.split('_')[1]
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        lower_black_t = np.array([0,0,0])
        upper_black_t = np.array([20,20,20])
        mask1 = cv2.inRange(hsv, lower_black_t, upper_black_t)

        li = []
        for line in mask1:
          li = line
          res = [i for i, val in enumerate(li) if val == 0]
          st, end = res[0], res[-1]
          for i in range(len(line)):
            if line[i] == 255:
              line[i] = 1
          for i in range(st, end):
            line[i] = 0

        conv_mask = np.array(format_mask(mask1))
        poly = binary_mask_to_polygon(conv_mask)

        px = [a for a in poly[0][0::2]]
        py = [a for a in poly[0][1::2]]
        poly2 = [[x, y] for x, y in zip(px, py)]
        poly2 = [p for x in poly2 for p in x]

        record = {}
        
        record["file_name"] = img_file
        record["height"] = height
        record["width"] = width
        record["image_id"] = idx
      
        objs = []
        
        obj = {
            "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [poly2],
            "category_id": classes.index(tclass),
            "iscrowd": 0
        }

        objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

joblib.dump(get_cow_dicts('/content/drive/MyDrive/KoreanBeef/data/00_source/images'), "/content/drive/MyDrive/KoreanBeef/data/pickledata.pkl")


100%|██████████| 10000/10000 [41:40<00:00,  4.00it/s]


['/content/drive/MyDrive/KoreanBeef/data/pickledata.pkl']

In [ ]:
joblib.load("/content/drive/MyDrive/KoreanBeef/data/pickledata.pkl")

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

for d in ['train']:
    DatasetCatalog.register("cow_" + d, lambda d=d: joblib.load("/content/drive/MyDrive/KoreanBeef/data/pickledata.pkl"))
    MetadataCatalog.get("cow_" + d).set(thing_classes=['1++', '1+', '1', '2', '3'])
cow_metadata = MetadataCatalog.get("cow_train")

### model_zoo
https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml"))
cfg.DATASETS.TRAIN = ("cow_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = len(parts_class)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml")

In [ ]:
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 10000   # 학습 횟수
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.TEST.EVAL_PERIOD = 100
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(parts_class)

model_dir = "faster_rcnn_R_50_C4_1x" # 모델이 저장 될 폴더이름
cfg.OUTPUT_DIR = '/content/drive/MyDrive/KoreanBeef/results/train/' + model_dir
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [
        #T.Resize((800,600)),
        #T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3),
        #T.RandomSaturation(0.8, 1.4),
        #T.RandomRotation(angle=[90, 90]),
        #T.RandomLighting(0.7),
        #T.RandomFlip(prob=0.3, horizontal=False, vertical=True),
    ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict
class AugTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)


#trainer = AugTrainer(cfg)

In [ ]:
trainer = DefaultTrainer(cfg)  
trainer.resume_or_load(resume=False)
trainer.train()

[06/18 12:32:27 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
model_final_4f86c3.pkl: 690MB [00:16, 42.8MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (6, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (6,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.b

[06/18 12:33:08 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[06/18 12:33:20 d2.utils.events]:  eta: 1:30:48  iter: 19  total_loss: 2.676  loss_cls: 1.702  loss_box_reg: 0.09902  loss_mask: 0.6854  loss_rpn_cls: 0.02401  loss_rpn_loc: 0.2095  time: 0.5456  data_time: 0.0362  lr: 3.9962e-05  max_mem: 4084M
[06/18 12:33:31 d2.utils.events]:  eta: 1:29:18  iter: 39  total_loss: 1.446  loss_cls: 0.4879  loss_box_reg: 0.09655  loss_mask: 0.6377  loss_rpn_cls: 0.02158  loss_rpn_loc: 0.1772  time: 0.5476  data_time: 0.0111  lr: 7.9922e-05  max_mem: 4084M
[06/18 12:33:42 d2.utils.events]:  eta: 1:30:26  iter: 59  total_loss: 0.9016  loss_cls: 0.1213  loss_box_reg: 0.08329  loss_mask: 0.5191  loss_rpn_cls: 0.01525  loss_rpn_loc: 0.1628  time: 0.5509  data_time: 0.0097  lr: 0.00011988  max_mem: 4084M
[06/18 12:33:53 d2.utils.events]:  eta: 1:30:15  iter: 79  total_loss: 0.7342  loss_cls: 0.1022  loss_box_reg: 0.08941  loss_mask: 0.3686  loss_rpn_cls: 0.008215  loss_rpn_loc: 0.154  time: 0.5527  data_time: 0.0094  lr: 0.00015984  max_mem: 4084M
[06/18 12:3

## Predict

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
cfg.TEST.DETECTIONS_PER_IMAGE = 1 # 한 이미지 당 예측 갯수
cfg.DATASETS.TEST = ()
predictor = DefaultPredictor(cfg)

[06/18 13:56:06 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                        | Shapes                                          |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.res2.0.conv1.*                         | backbone.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.res2.0.conv2.*                         | backbone.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.res2.0.conv3.*                         | backbone.res2.0.conv3.{norm.bias,norm.running_mean,norm.running_var,norm.weight,w

## Visualizer

In [ ]:
im = cv2.imread('/content/drive/MyDrive/KoreanBeef/data/01_splitdataset/val/images/cow_1++_100.jpg')
outputs = predictor(im)
outputs

{'instances': Instances(num_instances=1, image_height=248, image_width=200, fields=[pred_boxes: Boxes(tensor([[  3.0759,   3.0371, 195.9613, 242.3694]], device='cuda:0')), scores: tensor([0.3228], device='cuda:0'), pred_classes: tensor([0], device='cuda:0'), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]]], device='cuda:0')])}

In [ ]:
import random
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_cow_dicts('/content/drive/MyDrive/KoreanBeef/data/01_splitdataset/' + 'val' +'/images')
for d in random.sample(dataset_dicts, 3):
  
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)

  v = Visualizer(im[:, :, ::-1],
                  metadata=cow_metadata, 
                  scale=0.8, 
                  #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
  )
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  v.numpy(v)[0]
  plt.figure(figsize = (14, 10))
  plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
  plt.show()

100%|██████████| 2000/2000 [08:25<00:00,  3.96it/s]


AttributeError: ignored

In [ ]:
model_dir = "mask_rcnn_R_50_DC5_1x" # 모델이 저장 될 폴더이름
cfg.PREDOUTPUT_DIR = '/content/drive/MyDrive/KoreanBeef/results/predict/' + model_dir
os.makedirs(cfg.PREDOUTPUT_DIR, exist_ok=True)

y_preds = []
filenames = []

for filename in tqdm([file for file in os.listdir('/content/drive/MyDrive/KoreanBeef/data/01_splitdataset/test/images') if file.endswith('.jpg')]):
  im = cv2.imread(os.path.join('/content/drive/MyDrive/KoreanBeef/data/01_splitdataset/test/images', filename))
  outputs = predictor(im)
  grade = parts_class[outputs['instances'].pred_classes.to('cpu').numpy()[0]]
  filenames.append(filename)
  y_preds.append(grade)

label_decoding = {0:'1++', 1:'1+', 2:'1', 3:'2', 4:'3'}
pred_df = pd.DataFrame(list(zip(filenames, y_preds)), columns=['id','grade'])
pred_df['grade'] = pred_df['grade'].replace(label_decoding)


# Reorder 
sample_df = pd.read_csv('/content/drive/MyDrive/KoreanBeef/data/sample_submission.csv')
sorter = list(sample_df['id'])
resdf = pred_df.set_index('id')
result = resdf.loc[sorter].reset_index()

# Save predictions
resultpath = os.path.join(cfg.PREDOUTPUT_DIR, 'predictions.csv')
result.to_csv(resultpath, index=False)
print('Done')

100%|██████████| 8658/8658 [19:08<00:00,  7.54it/s]


Done


## Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir='/content/drive/MyDrive/KoreanBeef/hslee'

## 제출